In [ ]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey
from sqlalchemy.orm import sessionmaker, relationship, declarative_base, joinedload
from sqlalchemy import func

In [ ]:
Base = declarative_base()


class Person(Base):
    __tablename__ = "person"

    id = Column(Integer, primary_key=True)
    name = Column(String)
    age = Column(Integer)
    addresses = relationship("Address", back_populates="person")


class Address(Base):
    __tablename__ = "address"

    id = Column(Integer, primary_key=True)
    street = Column(String)
    city = Column(String)
    state = Column(String)
    person_id = Column(Integer, ForeignKey("person.id"))
    person = relationship("Person", back_populates="addresses")

In [ ]:
engine = create_engine("sqlite:///b2_two_tables.db", echo=True)
Session = sessionmaker(bind=engine)
session = Session()

In [1]:
# 1. Get all people and their associated addresses

people = session.query(Person).options(joinedload(Person.addresses)).all()

# 2. Get all people and their primary address (the first one listed)
people = (
    session.query(Person, Address)
    .filter(
        Person.id == Address.person_id,
        Address.id
        == session.query(func.min(Address.id)).filter(Address.person_id == Person.id),
    )
    .all()
)

# 3. Get all people and their primary address using a subquery
subq = (
    session.query(Person.id, func.min(Address.id).label("min_id"))
    .outerjoin(Address)
    .group_by(Person.id)
    .subquery()
)
people = (
    session.query(Person, Address)
    .filter(Person.id == Address.person_id, Address.id == subq.c.min_id)
    .all()
)

# 4. Get all people who live in a specific city
people = session.query(Person).join(Address).filter(Address.city == "New York").all()

# 5. Get all people who live in a specific state and are over the age of 25
people = (
    session.query(Person)
    .join(Address)
    .filter(Address.state == "California", Person.age > 25)
    .all()
)

# 6. Get all people and their associated addresses sorted by city
people = (
    session.query(Person)
    .options(joinedload(Person.addresses))
    .join(Address)
    .order_by(Address.city.asc())
    .all()
)

# 7. Get all people and their associated addresses sorted by city and street
people = (
    session.query(Person)
    .options(joinedload(Person.addresses))
    .join(Address)
    .order_by(Address.city.asc(), Address.street.asc())
    .all()
)

# 8. Get all people and their associated addresses, but exclude those who live on 'Main St'
people = (
    session.query(Person)
    .options(joinedload(Person.addresses))
    .join(Address)
    .filter(Address.street != "Main St")
    .all()
)

# 9. Get the total number of people in the database
num_people = session.query(func.count(Person.id)).scalar()

# 10. Get the average age of all people in the database
avg_age = session.query(func.avg(Person.age)).scalar()

# 11. Get the names and ages of all people who live in a specific state
people = (
    session.query(Person.name, Person.age)
    .join(Address)
    .filter(Address.state == "Texas")
    .all()
)

# 12. Get the number of people who live in each city
city_count = (
    session.query(Address.city, func.count(Person.id))
    .join(Person)
    .group_by(Address.city)
    .all()
)

# 13. Get the number of people who live in each state
state_count = (
    session.query(Address.state, func.count(Person.id))
    .join(Person)
    .group_by(Address.state)
    .all()
)


# 14. Get the name and age of the youngest person in the database who lives in a specific city
youngest_person = (
    session.query(Person.name, Person.age)
    .join(Address)
    .filter(Address.city == "Seattle")
    .order_by(Person.age.asc())
    .first()
)

# 15. Get the name and age of the oldest person in the database who lives in a specific state
oldest_person = (
    session.query(Person.name, Person.age)
    .join(Address)
    .filter(Address.state == "Florida")
    .order_by(Person.age.desc())
    .first()
)

# 16. Get all people and their associated addresses with an address in a specific state or city
people = (
    session.query(Person)
    .options(joinedload(Person.addresses))
    .join(Address)
    .filter(or_(Address.state == "California", Address.city == "New York"))
    .all()
)

# 17. Get all people and their associated addresses where the address street contains the word "Main"
people = (
    session.query(Person)
    .options(joinedload(Person.addresses))
    .join(Address)
    .filter(Address.street.like("%Main%"))
    .all()
)

# 18. Get all people and their associated addresses where the address street starts with "Main"
people = (
    session.query(Person)
    .options(joinedload(Person.addresses))
    .join(Address)
    .filter(Address.street.startswith("Main"))
    .all()
)

# 19. Get all people and their associated addresses where the address street ends with "Avenue"
people = (
    session.query(Person)
    .options(joinedload(Person.addresses))
    .join(Address)
    .filter(Address.street.endswith("Avenue"))
    .all()
)

# 20. Get all people and their associated addresses where the address street is either "Main St" or "Broadway"
people = (
    session.query(Person)
    .options(joinedload(Person.addresses))
    .join(Address)
    .filter(Address.street.in_(["Main St", "Broadway"]))
    .all()
)

# 21. Get all people and their associated addresses where the address state is not "California"
people = (
    session.query(Person)
    .options(joinedload(Person.addresses))
    .join(Address)
    .filter(Address.state != "California")
    .all()
)

# 22. Get all people and their associated addresses sorted by the length of their street name (shortest to longest)
people = (
    session.query(Person)
    .options(joinedload(Person.addresses))
    .join(Address)
    .order_by(func.length(Address.street))
    .all()
)

# 23. Get all people and their associated addresses sorted by the length of their street name (longest to shortest)
people = (
    session.query(Person)
    .options(joinedload(Person.addresses))
    .join(Address)
    .order_by(func.length(Address.street).desc())
    .all()
)

# 24. Get all people who have more than one address
people = (
    session.query(Person)
    .join(Address)
    .group_by(Person.id)
    .having(func.count(Address.id) > 1)
    .all()
)

# 25. Get all people and their associated addresses sorted by their age (oldest to youngest)
people = (
    session.query(Person)
    .options(joinedload(Person.addresses))
    .order_by(Person.age.desc())
    .all()
)

NameError: name 'session' is not defined